In [18]:
# Libraries
import json
import pymongo
import requests
from lxml import html
import re

In [19]:
# Database connection
""" in case you want to create your on database, just change this 3 variables according to your database. 
Database and collections will be created automatically."""
username = "globaladmin"
password = "UEKaeSc8Q3LK5naw"
db_url = "cluster1.etdy2wm.mongodb.net"

# database string
CNX_STR = f"mongodb+srv://{username}:{password}@{db_url}/?connectTimeoutMS=50000"
client = pymongo.MongoClient(CNX_STR)

# heck if the database exists
if "manage2sail" in client.list_database_names():
    print("Database exists!")
    db = client.manage2sail
else:
    db = client["manage2sail"]
    print("Database created!")



Database exists!


In [20]:
# scrape the data
base_url = 'https://www.manage2sail.com/en-US/search'
params = {
    'filterYear': '2023',
    'filterMonth': '',
    'filterCountry': 'SUI',
    'filterRegion': 'e3165406-2424-470e-9ec2-d9dac6c9b742',
    'filterClass': '',
    'filterClubId': '',
    'filterScoring': '',
    'paged': 'false',
    'filterText': ''
}

all_data = []
response = requests.get(base_url, params=params)

if response.status_code == 200:
    # Parse the HTML content
    tree = html.fromstring(response.content)
    table = tree.xpath('/html/body/div[2]/div[3]/div[2]/div/table')[0]
    data = []

    for row in table.xpath('./tbody/tr'):
        row_data = [cell.text_content().strip() for cell in row.xpath('./td')]
        link = row.xpath('./td[4]/a/@href')
        row_data.append(link[0] if link else None)
        data.append(row_data)

    all_data.extend(data)
else:
    print(f"Failed to retrieve page")

# print(all_data)

formatted_data = [
    {
        'year': row[0],
        'startdate': row[1],
        'enddate': row[2],
        'nameevent': row[3],
        'status': row[4],
        'nation': row[5],
        'city': row[6],
        'hostclub': row[7],
        'link': row[8]
    } for row in all_data
]

# To visualize the formatted data
print(formatted_data[:5])  # Print the first 5 for brevity

# Save formatted data as JSON
#with open('manage2sail_eventlist.json', 'w') as f:
#   json.dump(formatted_data, f, indent=4)

print("Formatted data saved as manage2sail_eventlist.json")

[{'year': '2023', 'startdate': '23.01.', 'enddate': '23.01.', 'nameevent': 'Wettfahrtregeln - Kurs I - Teil I\nS', 'status': 'Finished', 'nation': 'SUI', 'city': 'Zürich', 'hostclub': 'Zürichsee-Segler-Verband', 'link': '/en-US/event/232096ab-a96f-48e3-b91e-ff1249d40c2a'}, {'year': '2023', 'startdate': '30.01.', 'enddate': '30.01.', 'nameevent': 'Wettfahrtregeln - Kurs I - Teil II\nS', 'status': 'Finished', 'nation': 'SUI', 'city': 'Zürich', 'hostclub': 'Zürichsee-Segler-Verband', 'link': '/en-US/event/f2e3b835-51dd-4d91-9070-7010ec21f847'}, {'year': '2023', 'startdate': '31.01.', 'enddate': '31.01.', 'nameevent': 'Basiskurs für Offizielle - Kurs II - Teil I\nS', 'status': 'Finished', 'nation': 'SUI', 'city': 'Zürich', 'hostclub': 'Zürichsee-Segler-Verband', 'link': '/en-US/event/38e912b5-fb9d-4263-b61c-060e996b8240'}, {'year': '2023', 'startdate': '06.02.', 'enddate': '06.02.', 'nameevent': 'Basiskurs für Offizielle - Kurs II - Teil II\nS', 'status': 'Finished', 'nation': 'SUI', 'city

In [21]:
#Load data to db
db.events.insert_many(formatted_data)

InsertManyResult([ObjectId('66374809fa39d5c02e09cac4'), ObjectId('66374809fa39d5c02e09cac5'), ObjectId('66374809fa39d5c02e09cac6'), ObjectId('66374809fa39d5c02e09cac7'), ObjectId('66374809fa39d5c02e09cac8'), ObjectId('66374809fa39d5c02e09cac9'), ObjectId('66374809fa39d5c02e09caca'), ObjectId('66374809fa39d5c02e09cacb'), ObjectId('66374809fa39d5c02e09cacc'), ObjectId('66374809fa39d5c02e09cacd'), ObjectId('66374809fa39d5c02e09cace'), ObjectId('66374809fa39d5c02e09cacf'), ObjectId('66374809fa39d5c02e09cad0'), ObjectId('66374809fa39d5c02e09cad1'), ObjectId('66374809fa39d5c02e09cad2'), ObjectId('66374809fa39d5c02e09cad3'), ObjectId('66374809fa39d5c02e09cad4'), ObjectId('66374809fa39d5c02e09cad5'), ObjectId('66374809fa39d5c02e09cad6'), ObjectId('66374809fa39d5c02e09cad7'), ObjectId('66374809fa39d5c02e09cad8'), ObjectId('66374809fa39d5c02e09cad9'), ObjectId('66374809fa39d5c02e09cada'), ObjectId('66374809fa39d5c02e09cadb'), ObjectId('66374809fa39d5c02e09cadc'), ObjectId('66374809fa39d5c02e09ca

In [22]:
# remove all events that are seminar (eventname contains "\nS$")
collection = db.events
filter_criteria = {"nameevent": { "$regex": "\nS$", "$options": "i" }}
result = collection.delete_many(filter_criteria)

deleted_count = result.deleted_count
print(f"{deleted_count} documents with nameevent containing '\s' (seminar)  have been deleted.")

8 documents with nameevent containing '\s' (seminar)  have been deleted.


In [23]:
# remove all events that contain "jeden" in their nameevent inidactin a weekliy recuring event
collection = db.events
pipeline = [
    {"$match": {"nameevent": {"$regex": "jeden", "$options": "i"}}},
    {"$group": {"_id": None, "ids": {"$push": "$_id"}}}
]

result = collection.aggregate(pipeline)
deleted_count = 0
for doc in result:
    if "ids" in doc:
        deleted_count = len(doc["ids"])
        collection.delete_many({"_id": {"$in": doc["ids"]}})

print(f"{deleted_count} documents with eventname containing 'jeden' have been deleted.")

2 documents with eventname containing 'jeden' have been deleted.


In [24]:
# delete all events with status "gelöscht" (deleted events)
collection = db.events
pipeline = [
    {"$match": {"$and": [
        {"status": "gelöscht"}
    ]}},
    {"$group": {"_id": None, "ids": {"$push": "$_id"}}}
]

result = collection.aggregate(pipeline)
deleted_count = 0
for doc in result:
    if "ids" in doc:
        deleted_count = len(doc["ids"])
        collection.delete_many({"_id": {"$in": doc["ids"]}})

print(f"{deleted_count} documents with status 'gelöscht' have been deleted.")

0 documents with status 'gelöscht' have been deleted.


In [25]:
# clean hostclub column 
def clean_spaces(text):
    # remove " Zürichsee-Segler-Verband" if present
    text = text.replace("Zürichsee-Segler-Verband", "")
    # replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # remove leading and trailing spaces
    text = text.strip()
    return text

collection = db.events
updated_count = 0  

for document in collection.find():
    if 'hostclub' in document:
        # Clean the hostclub field
        cleaned_hostclub = clean_spaces(document['hostclub'])
        # Update the document in the database
        update_result = collection.update_one({'_id': document['_id']}, {'$set': {'hostclub': cleaned_hostclub}})
        if update_result.modified_count > 0:
            updated_count += 1

print(f"{updated_count} documents have been updated - hostclub column cleaned.")

9 documents have been updated - hostclub column cleaned.


In [26]:
# reformat the date columns
import datetime # otherwise throws an error
collection = db.events
updated_count = 0

for document in collection.find():
    # check if fields are present
    if 'year' in document and 'startdate' in document and 'enddate' in document:
        # Construct full date strings
        full_startdate_str = document['startdate'] + " " + str(int(document['year']))
        full_enddate_str = document['enddate'] + " " + str(int(document['year']))
        
        # convert 
        startdate = datetime.datetime.strptime(full_startdate_str, "%d.%m. %Y")
        enddate = datetime.datetime.strptime(full_enddate_str, "%d.%m. %Y")
        
        # update the document in the database with the formatted dates
        update_result = collection.update_one(
            {'_id': document['_id']},
            {'$set': {
                'iso_startdate': startdate.isoformat(),
                'iso_enddate': enddate.isoformat(),
                'startdate': startdate,
                'enddate': enddate
            }}
        )
        
        if update_result.modified_count > 0:
            updated_count += 1

print(f"{updated_count} documents have been formatted and updated.")

65 documents have been formatted and updated.


In [27]:
# remove column 'year' from all documents
collection = db.events

updated_count = 0
for document in collection.find():
    # check if 'year' field is present
    if 'year' in document:
        # update document
        update_result = collection.update_one(
            {'_id': document['_id']},
            {'$unset': {'year': ''}}
        )

        if update_result.modified_count > 0:
            updated_count += 1

print(f"{updated_count} documents have been updates to remove ")

65 documents have been updates to remove 


In [28]:
# add prefix to all link, so it is complete
collection = db.events
updated_count = 0

for document in collection.find():
    # check if the document has a 'link' field
    if 'link' in document:
        # "https://manage2sail.com" as prefix of each link
        updated_link = "https://manage2sail.com" + document['link']
        
        # update the document
        update_result = collection.update_one(
            {'_id': document['_id']},
            {'$set': {'link': updated_link}}
        )
        
        if update_result.modified_count > 0:
            updated_count += 1

print(f"{updated_count} documents have been updated with the link prefix.")

65 documents have been updated with the link prefix.
